In [1]:
!sudo apt update
!sudo apt install gcc


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Get:14 http://secur

In [2]:
# import data from wget
!wget "https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/EncryptSHA512/EncryptSHA512.c"

--2024-10-21 09:18:20--  https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/EncryptSHA512/EncryptSHA512.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214 [text/plain]
Saving to: ‘EncryptSHA512.c’

EncryptSHA512.c     100%[===================>]     214  --.-KB/s    in 0s      

2024-10-21 09:18:20 (3.36 MB/s) - ‘EncryptSHA512.c’ saved [214/214]



In [3]:
!gcc EncryptSHA512.c -o encrypt -lcrypt


In [4]:
!./encrypt AA99

$6$AS$HMEdiidHnxIA3SMWFUSqvm0YoM1MqcPUZhxXhyCW7MERgBEC3kTBN0hZgzrzgbgAfREPvnTAmrTJfGnhOrZWc1


In [5]:
%%writefile CrackAZ99.c
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <crypt.h>
#include <unistd.h>
#include <stdbool.h>

int count = 0; // A counter used to track the number of combinations explored so far
bool password_found = false; // Flag to indicate if the password has been found

void substr(char *dest, char *src, int start, int length) {
    memcpy(dest, src + start, length);
    *(dest + length) = '\0';
}

void crack(char *salt_and_encrypted, int start, int end) {
    int x, y, z;
    char salt[7];
    char plain[7];
    char *enc;

    substr(salt, salt_and_encrypted, 0, 6);

    for (x = 'A'; x <= 'Z'; x++) {
        for (y = 'A'; y <= 'Z'; y++) {
            for (z = start; z <= end; z++) {
                if (password_found) return; // Exit if password is already found

                sprintf(plain, "%c%c%02d", x, y, z);
                enc = (char *)crypt(plain, salt);
                count++;
                if (strcmp(salt_and_encrypted, enc) == 0) {
                    printf("#%-8d%s %s\n", count, plain, enc);
                    password_found = true; // Set the flag to true
                    return; // Exit as we found the password
                } else {
                    printf("%-8d%s %s\n", count, plain, enc); // When the password is not found
                }
            }
        }
    }
}

int main(int argc, char *argv[]) {
    if (argc != 4) {
        fprintf(stderr, "Usage: %s <encrypted_password> <total_combinations> <thread_count>\n", argv[0]);
        return 1;
    }

    char *encrypted_password = argv[1];
    long long total_combinations = atoll(argv[2]);
    int thread_count = atoi(argv[3]);

    // Calculate slices
    int sliceList[thread_count];
    int remainder = total_combinations % thread_count;

    // Fill standard slices
    for (int i = 0; i < thread_count; i++) {
        sliceList[i] = total_combinations / thread_count;
    }

    // Distribute remainder
    for (int j = 0; j < remainder; j++) {
        sliceList[j]++;
    }

    int startList[thread_count];
    int endList[thread_count];

    for (int k = 0; k < thread_count; k++) {
        startList[k] = (k == 0) ? 0 : endList[k - 1] + 1;
        endList[k] = startList[k] + sliceList[k] - 1;
    }

    // Launch threads or processes here
    for (int g = 0; g < thread_count; g++) {
        printf("Thread %d: start = %d  end = %d\n", g + 1, startList[g], endList[g]);
        crack(encrypted_password, startList[g], endList[g]); // Call crack for each slice
        if (password_found) break; // Stop further processing if password is found
    }

    printf("%d solutions explored\n", count);

    return 0;
}

Writing CrackAZ99.c


In [6]:
!gcc -pthread CrackAZ99.c -o crack -lcrypt


In [12]:
!./crack '$6$AS$HMEdiidHnxIA3SMWFUSqvm0YoM1MqcPUZhxXhyCW7MERgBEC3kTBN0hZgzrzgbgAfREPvnTAmrTJfGnhOrZWc1' 1000 3


Thread 1: start = 0  end = 333
1       AA00 $6$AS$wKDMKDtx/s3ILNkNaRNFIM0w81/weD1UZ8daNhbQBXuj8L.7OY4trHnSraeizmFYrMwjlb1uRTPxu20rqhmMn/
2       AA01 $6$AS$q8RLKgmlY0mkEfg/qZWj.4t9209zRMtD.eI4ipzNsAVvXDl8Qbj0JUmtt6SyPrwz9dOsjh3t5nGd5BRshe1io0
3       AA02 $6$AS$rAyQ4g7amb3ZN0BZ0UAY9s0wGIA2enhNK9jILLs9nYuaVt7Xw9MUsJr4jJq./T3FvYYd9BfH3gNBTykqKVtev.
4       AA03 $6$AS$pd1fyoLSlot6m/4YWPqc3jO1kEecXmcAy3WM8uEf2vI0glGDumXtdkihWzv.4KvKHmWrkU/VdnfHaAfMXzK7f0
5       AA04 $6$AS$ckgtvppuszJ5Pr/0mp5oBQrl/zsUoWBNyXs.N2OlQOcOptBxa9CLPwdkHsbQU9Ok6nsfixedKHwIjyf4EUQwt1
6       AA05 $6$AS$mOUEGkBkxmKR1pej2wqGfgaPcc/sfUS5xaAtWO7b.tsBzRP4a3Hy6AVdcS8KcuFBl/yKYKy.tWrNCqiHht9ki0
7       AA06 $6$AS$1.rleaK1WbJq.mDdM9RhVQ1/bRvDsfRhedARlbRnjjECbdn8sPxZqYwxYfGhnthC4Lo3hFAZfPAyCR3zafGEM0
8       AA07 $6$AS$zub4C8x8vm8m8SO5/V/Mre8ov2QZ9la6bntPZ0hBTlzlhlOs/Z6bfcJJoUVIiP30w1kMZwwFcMP4eWGBpj1Lg/
9       AA08 $6$AS$2s9lI1PNYlJWI8cEPvmhFCZpTEZgJ1GgSJveFlDKRdwLrOl3m8QhYXzlJ2jQDH8aIDY9j7Ps/SxJi0vC0Cq28/
10      AA09 $6